In [ ]:
import pandas as pd
import numpy as np
from google.colab import files
import io
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
uploaded = files.upload()
data = pd.read_csv(io.BytesIO(uploaded['youtube_video.csv']))

Saving youtube_video.csv to youtube_video (1).csv


UnicodeDecodeError: ignored

In [ ]:
train, test = train_test_split(data, test_size = 0.1, random_state = 55)

In [ ]:
train.head(5)

,video_id,published_at,channel_id,video_title,view_count,period_day,channel_title,subscriber_count
577,jyTnRRM2nv4,2022-10-16 12:15,UCPVENfTGbgD-CUuvrN7rRGQ,5성급 호텔뷔페 무한리필 먹방 제주도 브이로그 1탄 korean mukbang ea...,168561,200,상해기SangHyuk,1130000
1097,yGtAFLs-oAM,2020-09-16 12:00,UCwsFLjczPWiiWBS9_cdSQIQ,엽기떡볶이 허니콤보 치킨 먹방ASMR MUKBANG SPICY TTEOKBOKKI ...,3443182,960,교광TV,3790000
1146,zNQO6tIFkaA,2018-01-29 19:17,UC7bRo3yLjtrRV9qU_KsE6nA,말수줄이고 국밥에만 집중 먹방Mukbang eating show,682861,1921,가르마[GARMA],1250000
913,SY4qWxzmvA0,2023-03-06 11:58,UCmE4NuTfSISuz6UFiktosag,대구 먹방 브이로그?? 강철 체력?? 소주 10병 들이붓고 등산에 해장술 까지?? ...,63858,59,세라는 술말려,61200
886,S7DecJ3Tejs,2022-11-18 13:00,UCKDryfi71TjzKFkr9J9wzew,국밥 먹방 이게 찐 리얼 돼지국밥 3KG + 전라도김치 순천 건봉국밥 먹방,16032,167,MUKBANG NEUNG 만능방송,87900


In [ ]:
def make_dataset(data):
  x_image = np.zeros((1,10800))
  for id in data['video_id']:
    image_file_path = '/content/drive/MyDrive/thumbnail_default/default_{0}.jpg'.format(id) #구글 드라이브 폴더에서 썸네일 이미지 불러옴
    image = Image.open(image_file_path).convert("L") #썸네일 이미지 그레이스케일로 변환
    image = np.array(image).reshape(1,10800) #flatten
    x_image = np.concatenate((x_image, image), axis=0)
  x_image = np.delete(x_image, 0, axis=0)
  x_metadata = data[['period_day', 'subscriber_count']].to_numpy()
  x_data = np.concatenate((x_image, x_metadata), axis=1) #x_data = x_image + x_metadata
  y_data = np.log10(data['view_count'].to_numpy()) #y_data에 해당하는 조회수를 로그 스케일로 변환
  print('dataset size :', len(x_data))
  print('dataset shape : ', x_data.shape)
  return x_data, y_data

In [ ]:
train_dataset_x, train_dataset_y = make_dataset(train)
test_dataset_x, test_dataset_y = make_dataset(test)

dataset size : 1042
dataset shape :  (1042, 10802)
dataset size : 116
dataset shape :  (116, 10802)


In [ ]:
model = LinearRegression()
model.fit(train_dataset_x, train_dataset_y)

LinearRegression()

In [ ]:
#train
print('MSE on train')
print(mean_squared_error(model.predict(train_dataset_x), train_dataset_y))

#test
print('MSE on test')
print(mean_squared_error(model.predict(test_dataset_x), test_dataset_y))

MSE on train
1.1949487207265905e-22
MSE on test
0.6136897741310471
